# Comprehensive Analysis and Trend Evaluation of Medical Device Recalls

## Dataset merging and cleaning notebook

### Hunter Belous, Jared Fox, Asad Kamal

In [1]:
import pandas as pd

In [2]:
# Load the datasets.

events_df = pd.read_csv(r'C:\Users\akama\Documents\SIADS593\events-1681209680.csv', low_memory=False)
devices_df = pd.read_csv(r'C:\Users\akama\Documents\SIADS593\devices-1681209661.csv', low_memory=False)
manufacturers_df = pd.read_csv(r'C:\Users\akama\Documents\SIADS593\manufacturers-1681209657.csv', low_memory=False)

In [3]:
# Check to see if datasets loaded correctly.

print(events_df.head())
print(devices_df.head())
print(manufacturers_df.head())

   id action action_classification action_level action_summary  \
0   1    NaN                   NaN          NaN         Recall   
1   2    NaN                   NaN          NaN         Recall   
2   3    NaN                   NaN          NaN         Recall   
3   4    NaN                   NaN          NaN         Recall   
4   5    NaN                   NaN          NaN         Recall   

            authorities_link country create_date data_notes        date  ...  \
0  https://www.titck.gov.tr/     TUR         NaN        NaN  2016-04-04  ...   
1  https://www.titck.gov.tr/     TUR         NaN        NaN  2016-09-20  ...   
2  https://www.titck.gov.tr/     TUR         NaN        NaN  2016-05-09  ...   
3  https://www.titck.gov.tr/     TUR         NaN        NaN  2016-12-19  ...   
4  https://www.titck.gov.tr/     TUR         NaN        NaN  2017-02-20  ...   

  status target_audience                 type  \
0    NaN             NaN  Field Safety Notice   
1    NaN             NaN

In [4]:
# Find common key in each dataset to merge on.

print("Events Dataset Columns:", events_df.columns.tolist())
print("Devices Dataset Columns:", devices_df.columns.tolist())
print("Manufacturers Dataset Columns:", manufacturers_df.columns.tolist())

Events Dataset Columns: ['id', 'action', 'action_classification', 'action_level', 'action_summary', 'authorities_link', 'country', 'create_date', 'data_notes', 'date', 'date_initiated_by_firm', 'date_posted', 'date_terminated', 'date_updated', 'determined_cause', 'documents', 'icij_notes', 'number', 'reason', 'source', 'status', 'target_audience', 'type', 'uid', 'uid_hash', 'url', 'slug', 'device_id', 'created_at', 'updated_at']
Devices Dataset Columns: ['id', 'classification', 'code', 'description', 'distributed_to', 'implanted', 'name', 'number', 'quantity_in_commerce', 'risk_class', 'slug', 'country', 'manufacturer_id', 'created_at', 'updated_at']
Manufacturers Dataset Columns: ['id', 'address', 'comment', 'name', 'parent_company', 'representative', 'slug', 'source', 'created_at', 'updated_at']


In [5]:
# Merge events_df with devices_df on 'device_id'.
# Using a left join to ensure that all records from the primary dataset are kept and be able to add information from
# secondary datasets without losing any pf the primary dataset's records.

merged_df1 = pd.merge(events_df, devices_df, left_on='device_id', right_on='id', how='left')

In [6]:
# Merge the resulting dataframe with manufacturers_df on 'manufacturer_id'.

final_df = pd.merge(merged_df1, manufacturers_df, left_on='manufacturer_id', right_on='id', how='left')

In [7]:
# Inspect the final merged dataset.

pd.set_option('display.max_columns', None)

print(final_df.head())
print("Shape of the final merged dataset:", final_df.shape)

   id_x action action_classification action_level action_summary  \
0     1    NaN                   NaN          NaN         Recall   
1     2    NaN                   NaN          NaN         Recall   
2     3    NaN                   NaN          NaN         Recall   
3     4    NaN                   NaN          NaN         Recall   
4     5    NaN                   NaN          NaN         Recall   

            authorities_link country_x create_date data_notes        date  \
0  https://www.titck.gov.tr/       TUR         NaN        NaN  2016-04-04   
1  https://www.titck.gov.tr/       TUR         NaN        NaN  2016-09-20   
2  https://www.titck.gov.tr/       TUR         NaN        NaN  2016-05-09   
3  https://www.titck.gov.tr/       TUR         NaN        NaN  2016-12-19   
4  https://www.titck.gov.tr/       TUR         NaN        NaN  2017-02-20   

  date_initiated_by_firm date_posted date_terminated date_updated  \
0                    NaN         NaN             NaN       

In [8]:
# Check to see if everything merged properly.

print(f"events_df shape: {events_df.shape}")
print(f"devices_df shape: {devices_df.shape}")
print(f"manufacturers_df shape: {manufacturers_df.shape}")
print(f"merged_df1 shape: {merged_df1.shape}")
print(f"final_df shape: {final_df.shape}")

events_df shape: (124969, 30)
devices_df shape: (118249, 15)
manufacturers_df shape: (31827, 10)
merged_df1 shape: (124969, 45)
final_df shape: (124969, 55)


### Data Cleaning Steps

In [9]:
# Drop duplicates.

final_df.drop_duplicates(inplace=True)

In [10]:
# Convert 'date' column to datetime.

final_df['date'] = pd.to_datetime(final_df['date'], errors='coerce')

In [11]:
# Rename columns to a consistent format.

final_df.columns = final_df.columns.str.lower().str.replace(' ', '_')

#### Handling Missing Values

There seems to be quite a substantial amount of missing values in the final merged dataset. Simply dropping every record with a missing value would not be conducive to our comprehensive analysis and trend evaluation of medical device recalls. This would result in a great loss of data and information.

The approach to handling missing values should be carefully considered to ensure that the analysis remains robust and meaningful. We need a strategic imputation method to deal with the missing values in the context of our project.

In [12]:
# How many missing values in each column are we dealing with?

missing_values_before = final_df.isnull().sum()
print(missing_values_before)

id_x                           0
action                     62488
action_classification      72019
action_level              122513
action_summary             95864
authorities_link              82
country_x                      1
create_date               119768
data_notes                 67041
date                       76209
date_initiated_by_firm     69311
date_posted                80782
date_terminated            93482
date_updated              124738
determined_cause           89151
documents                  83930
icij_notes                     1
number_x                   80958
reason                     59299
source_x                       1
status                     88440
target_audience           124529
type                           1
uid                        20682
uid_hash                   20682
url                        24720
slug_x                         0
device_id                      0
created_at_x                   0
updated_at_x                   0
id_y      

There are some features of the dataset that are essential for our analysis such as device_id, manufacturer_id, date, and reason. Other features add significant value such as classification, description, determined_cause, country_x. Some columns have large proportions of missing values and are less critical to the core analysis: action, action_classification, action_level, date_initiated_by_firm, documents, etc. For the remaining columns, we can use placeholders for these features that are nice to have but not essential (target_audience, authorities_link, etc).

In [13]:
# Drop rows where essential information is missing.

essential_columns = ['device_id', 'manufacturer_id', 'date', 'reason']
final_df.dropna(subset=essential_columns, inplace=True)

In [14]:
# Impute missing values for important categorical columns with the mode.

important_categorical_cols = ['classification', 'country_x', 'determined_cause', 'description', 'name_y']
for col in important_categorical_cols:
    if col in final_df.columns:
        mode_value = final_df[col].mode()
        if not mode_value.empty:
            final_df[col].fillna(mode_value[0], inplace=True)

In [15]:
# Drop columns with a large proportion of missing values.

columns_to_drop = [
    'action', 'action_classification', 'action_level', 'data_notes',
    'date_initiated_by_firm', 'date_posted', 'date_terminated', 'date_updated',
    'documents', 'address', 'comment', 'representative', 'create_date'
]

final_df.drop(columns=columns_to_drop, inplace=True)

In [16]:
# Impute less critical information with placeholders.

final_df.fillna({
    'target_audience': 'Unknown',
    'authorities_link': 'N/A',
    'status': 'Unknown',
    'distributed_to': 'N/A',
    'implanted': 'N/A',
    'number_x': 'N/A',
    'number_y': 'N/A',
    'quantity_in_commerce': 'N/A',
    'risk_class': 'N/A',     
    'action_summary': 'No Summary', 
    'uid': 'N/A',                    
    'uid_hash': 'N/A',              
    'url': 'No URL',                 
    'name_x': 'Unknown Device Name', 
    'name_y': 'Unknown Manufacturer Name', 
    'description': 'No Description',       
    'code': 'Unknown',                
    'determined_cause': 'Unknown Cause', 
    'parent_company': 'Unknown Parent Company' 
}, inplace=True)

In [17]:
# Check to see if there are any missing values after.

missing_values_after = final_df.isnull().sum()
print(missing_values_after)

id_x                    0
action_summary          0
authorities_link        0
country_x               0
date                    0
determined_cause        0
icij_notes              0
number_x                0
reason                  0
source_x                0
status                  0
target_audience         0
type                    0
uid                     0
uid_hash                0
url                     0
slug_x                  0
device_id               0
created_at_x            0
updated_at_x            0
id_y                    0
classification          0
code                    0
description             0
distributed_to          0
implanted               0
name_x                  0
number_y                0
quantity_in_commerce    0
risk_class              0
slug_y                  0
country_y               0
manufacturer_id         0
created_at_y            0
updated_at_y            0
id                      0
name_y                  0
parent_company          0
slug        

In [22]:
# Final DataFrame shape.

print(final_df.shape)

(9067, 42)


This approach, which may not best the best one out there, handles the columns with missing values by filling them with placeholders, ensuring that critical columns are not dropped unless necessary, and preserving the integrity of the dataset for further analysis. By doing this, our dataset becomes more manageable, complete, and ready for our respective analyses.

In [18]:
# Write the final merged DataFrame to an Excel file.

file_path = r'C:\Users\akama\Documents\SIADS593\dataset_merged.xlsx'
final_df.to_excel(file_path, index=False, engine='xlsxwriter')

In [5]:
# Record dependencies.

%load_ext watermark
%watermark -v -m -p matplotlib,numpy,pandas,seaborn

Python implementation: CPython
Python version       : 3.9.13
IPython version      : 8.8.0

matplotlib: 3.7.2
numpy     : 1.24.1
pandas    : 1.5.3
seaborn   : 0.12.2

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 5, GenuineIntel
CPU cores   : 16
Architecture: 64bit

